# stackoverflow

https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix

In [1]:
import numpy as np

In [2]:
arr= np.array([[ 1,  0,  0,  1,  1,  0],
   [ 1,  0,  0,  0,  0,  0],
   [ 1,  1,  1,  1,  0,  0],])

In [3]:
arr

array([[1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0]])

In [4]:
arr.itemsize

8

In [5]:
(arr[:, None, :] != arr).sum(2)

# array([[0, 2, 3],
#        [2, 0, 3],
#        [3, 3, 0]])
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

In [6]:
compute_HammingDistance(arr)

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

# use bool

https://stackoverflow.com/questions/42752610/python-how-to-generate-the-pairwise-hamming-distance-matrix

In [7]:
arr = arr.astype(bool)

In [8]:
arr

array([[ True, False, False,  True,  True, False],
       [ True, False, False, False, False, False],
       [ True,  True,  True,  True, False, False]])

In [9]:
arr.itemsize

1

In [10]:
(arr[:, None, :] != arr).sum(2)

# array([[0, 2, 3],
#        [2, 0, 3],
#        [3, 3, 0]])
def compute_HammingDistance(X):
    return (X[:, None, :] != X).sum(2)

In [11]:
compute_HammingDistance(arr)

array([[0, 2, 3],
       [2, 0, 3],
       [3, 3, 0]])

# setup

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata

BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [13]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# hash

In [14]:
def hamming(x1, x2): 
    x1,x2 = (bin(int(x, 16)) for x in (x1, x2))
    return sum(o1!=o2 for o1,o2 in zip(x1,x2))
n = 5
hdists = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(i, n):
        x1 = df.image_phash[i]
        x2 = df.image_phash[j]
        hdists[i,j]=hdists[j,i]=hamming(x1, x2)
hdists

100%|██████████| 5/5 [00:00<00:00, 6687.35it/s]


array([[ 0., 42., 40., 30., 32.],
       [42.,  0., 28., 28., 34.],
       [40., 28.,  0., 34., 44.],
       [30., 28., 34.,  0., 34.],
       [32., 34., 44., 34.,  0.]])

In [15]:
hfeats = df[:5].image_phash.apply(lambda x: bin(int(x, 16))).values

In [16]:
X = np.array([[eval(o) for o in x[2:]] for x in hfeats])
X

array([[1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

In [17]:
X = X.astype(bool)

In [18]:
compute_HammingDistance(X)

array([[ 0, 42, 40, 30, 32],
       [42,  0, 28, 28, 34],
       [40, 28,  0, 34, 44],
       [30, 28, 34,  0, 34],
       [32, 34, 44, 34,  0]])

# all data -- OOM on kaggle

# loop

In [45]:
n = 4
hfeats = df[:n].image_phash.apply(lambda x: bin(int(x, 16))).values
X = np.array([[eval(o) for o in x[2:]] for x in hfeats], dtype=np.bool)

In [46]:
compute_HammingDistance(X)

array([[ 0, 42, 40, 30],
       [42,  0, 28, 28],
       [40, 28,  0, 34],
       [30, 28, 34,  0]])

In [47]:
n = len(X)
for i in tqdm(range(n)):
    print((X[:, None, :]!=X[i]).sum(2))

100%|██████████| 4/4 [00:00<00:00, 2565.32it/s]

[[ 0]
 [42]
 [40]
 [30]]
[[42]
 [ 0]
 [28]
 [28]]
[[40]
 [28]
 [ 0]
 [34]]
[[30]
 [28]
 [34]
 [ 0]]


In [50]:
(X[:, None, :]!=X[i]).sum(2)[:,0]

array([ 0, 42, 40, 30])

In [51]:
n = len(X)
hdists = np.zeros((n, n), dtype=np.short)
for i in tqdm(range(n)):
    hdists[:,i] = ((X[:, None, :]!=X[i]).sum(2))[:,0]

100%|██████████| 4/4 [00:00<00:00, 15420.24it/s]


In [52]:
hdists

array([[ 0, 42, 40, 30],
       [42,  0, 28, 28],
       [40, 28,  0, 34],
       [30, 28, 34,  0]], dtype=int16)

# all data 

In [53]:
hfeats = df.image_phash.apply(lambda x: bin(int(x, 16))).values

In [54]:
%%time
X = np.array([[eval(o) for o in x[2:]] for x in hfeats], dtype=np.bool)


CPU times: user 8.29 s, sys: 0 ns, total: 8.29 s
Wall time: 8.29 s


In [55]:
%%time
n = len(X)
hdists = np.zeros((n, n), dtype=np.short)
for i in tqdm(range(n)):
    hdists[:,i] = ((X[:, None, :]!=X[i]).sum(2))[:,0]

100%|██████████| 34250/34250 [04:23<00:00, 129.85it/s]

CPU times: user 4min 23s, sys: 2.07 s, total: 4min 25s
Wall time: 4min 23s


# demo

In [56]:
hsims = 1/(1+hdists)

In [57]:
hsims

array([[1.        , 0.02325581, 0.02439024, ..., 0.02702703, 0.02857143,
        0.03225806],
       [0.02325581, 1.        , 0.03448276, ..., 0.03703704, 0.03030303,
        0.04347826],
       [0.02439024, 0.03448276, 1.        , ..., 0.03225806, 0.02564103,
        0.02857143],
       ...,
       [0.02702703, 0.03703704, 0.03225806, ..., 1.        , 0.03448276,
        0.03448276],
       [0.02857143, 0.03030303, 0.02564103, ..., 0.03448276, 1.        ,
        0.02857143],
       [0.03225806, 0.04347826, 0.02857143, ..., 0.03448276, 0.02857143,
        1.        ]])

# demo

In [76]:
i = np.random.choice(range(len(df)))
print(i)
df.iloc[i].to_dict()

8693


{'posting_id': 'train_3665527367',
 'image': '41506335d45d81352c40c6e28e71b96e.jpg',
 'image_phash': 'ef9190669c6e9634',
 'title': 'Pure Kids Fever Free Cooling Patch Gel isi 4',
 'label_group': 1715041013}

In [77]:
grp = df.iloc[i].label_group

df[df.label_group==grp][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
6677,Pure Kids Fever Free,e196ce6c939a6532,1715041013
8693,Pure Kids Fever Free Cooling Patch Gel isi 4,ef9190669c6e9634,1715041013
9930,Pure Baby & Kids Fever Free - 1 dus 4 lembar c...,b193ce249b9b6433,1715041013
11124,ORIGINAL Pure Kids Fever Free Isi 4 lembar coo...,80c53f327a236c7b,1715041013
11693,[BPOM] Pure Kids Fever Free Isi 4 lembar Cooli...,95b253cd508f6dc2,1715041013
14666,Purebaby Pure kids Fever Free / Plester Penuru...,b0cbce309b6434db,1715041013
16922,Pure Kids Fever Free isi 4pcs,a79e8c259bc864d3,1715041013
21976,Pure Kids Fever Free Cooling Patch [4 Pcs],d7e94cc660da2b70,1715041013
26413,BPOM - Pure Kids Fever Free Isi 4 lembar cooli...,94916b703a2f2f36,1715041013
27202,PURE KIDS FEVER FREE COOLING PATCH GEL ~ ISI 4...,e1966c32936b34cd,1715041013


In [78]:
ntop = 10

In [79]:
idx_sim = np.argsort(-hsims[i])[:ntop]
df.iloc[idx_sim][['title', 'image_phash', 'label_group']]

,title,image_phash,label_group
8693,Pure Kids Fever Free Cooling Patch Gel isi 4,ef9190669c6e9634,1715041013
10868,Felibite 500Gr / Makanan Kucing Kering Freshpa...,ed9398668e6dc630,2404669998
16570,Habbatussauda Extra Propolis Trigona 200 Kapsu...,eec59162946a976c,333597397
11052,Parfum Mobil Aroma Kopi Bali 100%,ee9191661e2cd5a5,1042780132
7314,Rak Sepatu Gantung karakter,e699996699669926,628283471
19895,LEM TOUCHSCREEN ZHANLIDA T7000 50ML MEDIUM VIS...,e699996699669926,3585316632
21113,Paseo Tissue Tissu Tisu Basah BABY WIPES isi 5...,ef11906e496b1795,2886450320
31765,SETELAN LIBBY PENDEK / SETELAN LIBBY BAGUS,eb909c72ce6e3684,2352043911
28502,Mangkok Masker Set / Mask Bowl masker Mangkok ...,ee95d82c9c6e1643,3211565621
15115,Rak sepatu karakter 9 susun,e699996699669926,628283471


# save 

In [62]:
pout = f"{HOME}/data/dev0011"
!mkdir -p $pout

In [63]:
np.save(f"{pout}/hdists.npy", hdists)
np.save(f"{pout}/hsims.npy", hsims)

# fin